In [1]:
# importar librerias
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import pandas as pd

In [2]:
#importar data
df_vts = pd.read_excel(r'./database/Ventas.xlsx', sheet_name = "Detalle" )
df_vts_acum = pd.read_excel(r'./database/Ventas.xlsx', sheet_name = "Acumulado")

In [3]:
#analisis de datos de la
df_vts.size

149265

In [4]:
df_vts_acum.size

16

In [5]:
df_vts.head(3)

,ID Pedido,Fecha compra,Modo envío,ID Cliente,ID Producto,Precio unitario,Cantidad,Importe,Segmento,País,Categoría,Sub-categoría,Nombre Producto,Coste Producción,Beneficio
0,CA-2016-152156,2020-11-08,Second Class,CG-12520,FUR-BO-10001798,261.960,2,523.920,Consumer,United States,Furniture,Bookcases,Bush Somerset Collection Bookcase,230.098156,63.723687
1,CA-2017-110198,2021-05-01,First Class,AG-10900,FUR-BO-10001798,314.352,3,943.056,Consumer,United States,Furniture,Bookcases,Bush Somerset Collection Bookcase,230.098156,252.761531
2,CA-2017-159793,2021-08-25,Standard Class,SV-20365,FUR-BO-10001798,130.980,2,261.960,Consumer,United States,Furniture,Bookcases,Bush Somerset Collection Bookcase,230.098156,-198.236313


In [6]:
df_vts.columns = ['idPed','fecha', 'mod_env','idCli','idPro','pre_unit','cant','imp','seg','pais','cat','sub_cat','pro','cost','ben']

In [7]:
df_vts.tail(3)

,idPed,fecha,mod_env,idCli,idPro,pre_unit,cant,imp,seg,pais,cat,sub_cat,pro,cost,ben
9948,CA-2017-113474,2021-03-30,First Class,TM-21490,OFF-EN-10004206,325.860,2,651.720,Consumer,Romania,Office Supplies,Envelopes,Multimedia Mailers,228.168887,195.382226
9949,CA-2016-160535,2020-11-25,Standard Class,BP-11050,TEC-MA-10004255,319.920,8,2559.360,Corporate,Romania,Technology,Machines,Konica Minolta magicolor 1690MF Multifunction ...,280.486707,315.466342
9950,CA-2016-129280,2020-05-03,First Class,SM-20905,TEC-MA-10003589,224.937,3,674.811,Consumer,Romania,Technology,Machines,Cisco 8961 IP Phone Charcoal,212.190300,38.240101


In [8]:
df_vts.dtypes

idPed               object
fecha       datetime64[ns]
mod_env             object
idCli               object
idPro               object
pre_unit           float64
cant                 int64
imp                float64
seg                 object
pais                object
cat                 object
sub_cat             object
pro                 object
cost               float64
ben                float64
dtype: object

In [9]:
#crear el objeto dash
app = dash.Dash

In [10]:
token = 'pk.eyJ1IjoibGVhZmFyMTk3NyIsImEiOiJjbDlyNzBuY28wMjRoM25vNHNzZDVsZ2x4In0.krumWnuj6VoVCzjgEb5Yrw'

In [11]:
#grafico geografico F 3
fig_map = go.Figure(go.Scattermapbox(
  lon = df_vts_acum['Longitud'],
  lat = df_vts_acum['Latitud'],
  text= df_vts_acum['Suma Ingresos'],
  marker=go.scattermapbox.Marker(
    size = df_vts_acum['Suma Ingresos']/50000,
    color = df_vts_acum['Suma Ingresos']/50000
  )
))

In [12]:
#grafico geografico F 3
fig_map.update_layout(
  autosize = True,
  hovermode = 'closest',
  mapbox = dict(
    accesstoken = token,
    bearing = 0,
    center = dict(
      lat = 40.41,
      lon = -3.7
    ),
    pitch = 0,
    zoom = 2
  )
)

In [13]:
#definicion del layout F1
app.layout = html.Div([
  html.Div([
  html.Label('País'),
  dcc.Dropdown(
    id = 'selector',
    options = [{'label':i, 'value':i} for i in df_vts['pais'].unique()],
    value = 'Spain'
  )
  ],
    style = {
      'width':'48%',
      'display':'inline-block'
    }
  ),
  html.Div([
    html.Label('Rango Fechas'),
    dcc.DatePickerRange(
      id = 'selector_date',
      start_date = df_vts['fecha'].min(),
      end_date = df_vts['fecha'].max()
    )
  ],
    style = {
      'width':'48%',
      'float':'right',
      'display':'inline-block'
    }
  ),
  #grafico (1x3)
  html.Div([
    dcc.Graph(
      id = 'barplot_vts'
    )
  ],
    style = {
      'width':'33%',
      'float':'left',
      'display':'inline-block'
    }
  ),
  html.Div([
    dcc.Graph(
      id = 'barplot_ben'
    )
  ],
    style = {
      'width':'33%',
      'float':'center',
      'display': 'inline-block'
    }
  ),
  html.Div([
    dcc.Graph( 
    id = 'lineplot_cant'
    )
  ],
    style = {
      'width':'33%',
      'float':'right',
      'display': 'inline-block'
    }
  ),
  #grafico geografico
  html.Div([
    dcc.Graph(
      id = 'map_vts',
      figure = fig_map
    )
  ],
    style = {
      'width':'100%'
    }
  )
])

In [ ]:
@app.callback(
  Output('barplot_vts', 'figure'),
  [
    Input('selector_date', 'start_date'),
    Input('selector_date', 'end_date'),
    Input('selector', 'value')
    ]
  )
def actualizar_graph_seg(fecha_min, fecha_max, seleccion):
  
    filtered_df = df_vts[(df_vts["fecha"]>=fecha_min) & (df_vts["fecha"]<=fecha_max) & (df_vts["pais"]==seleccion)]

# FASE 2: CREACIÓN DE GRÁFICOS Y GROUPBY
    df_agrupado = filtered_df.groupby("seg")["imp"].agg("sum").to_frame(name = "Ingresos").reset_index()

    return{
        'data': [go.Bar(x=df_agrupado["seg"],
                            y=df_agrupado["Ingresos"]
                            )],
        'layout': go.Layout(
            title="¿Cuáles han sido las ventas en cada segmento de clientes?",
            xaxis={'title': "Segmento"},
            yaxis={'title': "Ingresos totales"},
            hovermode='closest'
        )}


In [ ]:
#decoradores para interactividad F 5
@app.callback(
  Output('barplot_ben','figure'),
  [
    Input('selector_date','start_date'),
    Input('selector_date','end_date'),
    Input('selector','value'),
    Input('barplot_vts', 'hoverData')
  ]
)#funciones de interactivdad F. 4
def actualizar_graph(fmin, fmax, select, hoverdata):
  v_index = hoverdata['points'][0]['x']
  filtro = df_vts[(df_vts['fecha']>=fmin)&(df_vts['fecha']<=fmax)&(df_vts['pais']== select) & (df_vts['seg'] == v_index)]

  #Creacion de graficos 2 y groupby F.2
  df_group = filtro.groupby('cat')['ben'].agg('sum').to_frame('ben').reset_index()

  return {
    'data':[go.Bar(
      x = df_group['cat'],
      y = df_group['ben']
    )],
    'layout': go.Layout(
      title = '¿Cuales han sido los beneficios por categoria?',
      xaxis = {'title':'Categoría'},
      yaxis = {'title':'Beneficos Totales'},
      hovermode = 'closest'
    )
  }

In [ ]:
#decoradores para interactividad F 5
@app.callback(
  Output('lineplot_cant','figure'),
  [
    Input('selector_date','start_date'),
    Input('selector_date','end_date'),
    Input('selector','value'),
    Input('barplot_vts','hoverData')
  ]
)
def actualizar_graph(fmin, fmax, select, hoverdata):
  v_index = hoverdata['points'][0]['x']
  filtro = df_vts[(df_vts['fecha']>=fmin) & (df_vts['fecha']<=fmax) & (df_vts['pais']== select) & (df_vts['seg'] == v_index)]


  #Creacion de graficos 3 y groupby F.2
  df_group = filtro.groupby('fecha')['cant'].agg('sum').to_frame(name = 'cant').reset_index()

  return {
    'data': [go.Scatter(
      x = df_group['fecha'],
      y = df_group['cant'],
      mode = 'lines+markers'
    )],
    'layout': go.Layout(
      title = 'Cual es la evolución de la cantidad de pedidos solicitados?',
      xaxis = {'title':'Fecha'},
      yaxis = {'title':'Cantidad'},
      hovermode = 'closest'
    )
  }

In [ ]:
if __name__ == '__main__':
    app.run_server(port=7200)